이 노트북은 LangChain의 다양한 리트리버(임베딩 리트리버, BM25 리트리버, 앙상블 리트리버)의 성능을
비교하는 방법을 보여줍니다. 실제 문서에서 질문-문서 쌍을 생성하고, 다양한 검색 메트릭으로 리트리버의
성능을 평가합니다.

In [ ]:
!pip install langchain langchain-community pypdf python-dotenv openai sentence-transformers datasets tqdm chromadb rank_bm25

In [ ]:
# 필요한 라이브러리 임포트
import os
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from typing import Dict, List, Set, Tuple, Any
from openai import OpenAI

# LangChain 임포트
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

## 1. OpenAI API 키 설정 및 PDF 파일 다운로드

먼저 OpenAI API 키를 설정하고 평가에 사용할 PDF 파일의 URL을 지정합니다.

In [ ]:
# OpenAI API 키 설정
os.environ["OPENAI_API_KEY"] = "여러분의 Key 값"

In [ ]:
# PDF URL 설정 - 삼성 GSAT 문제집을 사용합니다
pdf_url = "https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf"

## 2. PDF 문서 로드 및 청킹 처리

PDF 파일을 로드하고 청크로 분할합니다. 각 청크는 나중에 검색의 기본 단위가 됩니다.
청크 크기와 겹침 정도를 조절할 수 있습니다.

In [ ]:
print("PDF 문서 로드 중...")
loader = PyPDFLoader(pdf_url)  # PyPDFLoader로 PDF 파일 로드
pages = loader.load_and_split()
print(f"PDF에서 {len(pages)}개의 페이지를 로드했습니다.")

# RecursiveCharacterTextSplitter로 청크로 분할
# chunk_size: 각 청크의 최대 문자 수
# chunk_overlap: 청크 간 겹치는 문자 수 (문맥 연속성 유지를 위해)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)
documents = text_splitter.split_documents(pages)

PDF 문서 로드 중...


PDF에서 27개의 페이지를 로드했습니다.


In [ ]:
# 첫번째 문서 출력
documents[0]

Document(metadata={'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)', 'creator': 'nPDF (pdftk 1.41)', 'creationdate': '2017-08-16T00:21:02-08:00', 'moddate': '2017-08-16T00:21:02-08:00', 'source': 'https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf', 'total_pages': 27, 'page': 0, 'page_label': '1'}, page_content='2\n01 삼성전자 기업분석\n(Samsung Electronics Co., Ltd)\nⅠ 기업 일반 \n1  기업개요\n1) 기업소개 \n본사주소 경기도 수원시 영통구 삼성로 129(매탄동 416)\n사업분야 삼성그룹의 대표 기업으로 휴대폰, 정보통신기기, 반도체, TV 등을 생산 판매하는 제조업체\n홈페이지 www.samsung.com/sec 구분 전기전자 대기업  \n설립일 1961년 07월 01일 대표이사 권오현 \n총자산1) 244조 매출액2) 200조\n임직원수 95,374명 \n∙ 1975년 1월 주식시장 상장\n∙ 1984년 2월 삼성전자공업주식회사->삼성전자주식회사로 사명 변경 \n∙ CE(Consumer Electronics), IM(Information technology & Mobile communications), DS(Device Solutions) \n3개의 부문으로 나누어 독립 경영.\n부문 제품\nCE TV, 모니터, 냉장고, 세탁기, 에어컨, 프린터, 의료기기 등')

In [ ]:
# 첫 번째 문서 확인
print("첫 번째 문서 내용 샘플:")
print(documents[0].page_content)
print('==' * 50)
print("문서 메타데이터:")
print(documents[0].metadata)

첫 번째 문서 내용 샘플:
2
01 삼성전자 기업분석
(Samsung Electronics Co., Ltd)
Ⅰ 기업 일반 
1  기업개요
1) 기업소개 
본사주소 경기도 수원시 영통구 삼성로 129(매탄동 416)
사업분야 삼성그룹의 대표 기업으로 휴대폰, 정보통신기기, 반도체, TV 등을 생산 판매하는 제조업체
홈페이지 www.samsung.com/sec 구분 전기전자 대기업  
설립일 1961년 07월 01일 대표이사 권오현 
총자산1) 244조 매출액2) 200조
임직원수 95,374명 
∙ 1975년 1월 주식시장 상장
∙ 1984년 2월 삼성전자공업주식회사->삼성전자주식회사로 사명 변경 
∙ CE(Consumer Electronics), IM(Information technology & Mobile communications), DS(Device Solutions) 
3개의 부문으로 나누어 독립 경영.
부문 제품
CE TV, 모니터, 냉장고, 세탁기, 에어컨, 프린터, 의료기기 등
문서 메타데이터:
{'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)', 'creator': 'nPDF (pdftk 1.41)', 'creationdate': '2017-08-16T00:21:02-08:00', 'moddate': '2017-08-16T00:21:02-08:00', 'source': 'https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf', 'total_pages': 27, 'page': 0, 'page_label': '1'}


## 3. 문서에 대한 질문 생성

각 문서에 대한 질문을 자동으로 생성하는 함수입니다.  
GPT-4o를 사용하여 문서 내용에 기반한 질문을 생성합니다.  
이 질문들은 나중에 리트리버를 평가하는 데 사용됩니다.

각 질문은 특정 문서와 연결되어 있으며, 해당 문서는 그 질문에 대한 '정답' 문서로 간주됩니다.  
이를 통해 리트리버가 질문과 관련된 문서를 얼마나 잘 찾는지 평가할 수 있습니다.

In [ ]:
def generate_query_doc_pairs(documents, num_questions_per_doc=2):
    """
    각 문서에 대한 질문과 정답 문서를 생성하는 함수

    Args:
        documents: 문서 리스트
        num_questions_per_doc: 문서 당 생성할 질문 수

    Returns:
        queries: 질문 ID를 키로, 질문 텍스트를 값으로 하는 딕셔너리
        corpus: 문서 ID를 키로, 문서 텍스트를 값으로 하는 딕셔너리
        relevant_docs: 질문 ID를 키로, 관련 문서 ID 집합을 값으로 하는 딕셔너리
    """
    # OpenAI 클라이언트 초기화
    client = OpenAI()

    # 결과를 저장할 딕셔너리 초기화
    all_queries = {}  # 질문 ID -> 질문 텍스트
    corpus = {}       # 문서 ID -> 문서 텍스트
    relevant_docs = {} # 질문 ID -> 관련 문서 ID 집합

    # 프롬프트 템플릿 설정
    prompt_template = """\
    다음은 참고할 내용입니다.

    ---------------------
    {context_str}
    ---------------------

    위 내용을 바탕으로 낼 수 있는 질문을 {num_questions_per_chunk}개 만들어주세요.
    질문만 작성하고 실제 정답이나 보기 등은 작성하지 않습니다.

    해당 질문은 본문을 볼 수 없다고 가정합니다.
    따라서 '위 본문을 바탕으로~' 라는 식의 질문은 할 수 없습니다.

    질문은 아래와 같은 형식으로 번호를 나열하여 생성하십시오.

    1. (질문)
    2. (질문)
    """

    # 각 문서에 대해 질문 생성
    for i, doc in enumerate(tqdm(documents)):
        doc_id = doc.metadata.get('id', f"doc_{i}")
        corpus[doc_id] = doc.page_content

        # 질문 생성 요청
        messages = [
            {"role": "system", "content": "You are a helpful assistant that generates questions based on provided content."},
            {"role": "user", "content": prompt_template.format(
                context_str=doc.page_content,
                num_questions_per_chunk=num_questions_per_doc
            )}
        ]

        try:
            # GPT-4o로 질문 생성
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
                temperature=0.7,
            )

            # 응답 처리
            result = response.choices[0].message.content.strip().split("\n")

            # 질문 추출
            questions = []
            for line in result:
                if line.strip():
                    parts = line.strip().split('. ', 1)
                    if len(parts) > 1:
                        questions.append(parts[1])
                    elif "?" in line:  # 질문이라고 판단되면 추가
                        questions.append(line)

            # 추출된 질문에 대해 관련 문서 설정
            for q_idx, question in enumerate(questions):
                if len(question) > 0:
                    query_id = f"q_{i}_{q_idx}"
                    all_queries[query_id] = question

                    if query_id not in relevant_docs:
                        relevant_docs[query_id] = set()
                    relevant_docs[query_id].add(doc_id)

        except Exception as e:
            print(f"문서 {doc_id}에 대한 질문 생성 중 오류 발생: {e}")

    return all_queries, corpus, relevant_docs

위 함수는 각 문서마다 다음과 같은 작업을 수행합니다:

1. 문서 내용을 GPT-4o에 전달하여 그 내용에 대한 질문을 생성하도록 요청합니다.
2. 생성된 질문을 파싱하여 질문 리스트를 만듭니다.
3. 각 질문에 고유 ID를 부여하고, 질문이 어떤 문서에서 생성되었는지 추적합니다.

결과적으로 다음과 같은 데이터 구조를 생성합니다:
- `queries`: 질문 ID를 키로, 질문 텍스트를 값으로 하는 딕셔너리
- `corpus`: 문서 ID를 키로, 문서 텍스트를 값으로 하는 딕셔너리
- `relevant_docs`: 질문 ID를 키로, 해당 질문과 관련된 문서 ID 집합을 값으로 하는 딕셔너리

예를 들어, "삼성전자의 사업 영역은 무엇인가?"라는 질문이 있다면, 이 질문이 생성된 문서(삼성전자에 대한 정보가 있는 문서)가 "정답" 문서로 간주됩니다.

In [ ]:
# 질의-문서 쌍 생성 실행
queries, corpus, relevant_docs = generate_query_doc_pairs(documents)
print(f"생성된 질문 수: {len(queries)}")

  0%|          | 0/71 [00:00<?, ?it/s]

생성된 질문 수: 142


In [ ]:
# 생성된 질문 몇 개 샘플 확인
print("\n생성된 질문 샘플:")
for i, (query_id, query_text) in enumerate(list(queries.items())[:3]):
    print(f"질문 {i+1}: {query_text}")
    related_doc_ids = list(relevant_docs[query_id])
    print(f"  관련 문서 ID: {related_doc_ids[0]}")
    print(f"  문서 내용 일부: {corpus[related_doc_ids[0]][:100]}...\n")


생성된 질문 샘플:
질문 1: 삼성전자의 설립일은 언제인가요?
  관련 문서 ID: doc_0
  문서 내용 일부: 2
01 삼성전자 기업분석
(Samsung Electronics Co., Ltd)
Ⅰ 기업 일반 
1  기업개요
1) 기업소개 
본사주소 경기도 수원시 영통구 삼성로 129(매탄동...

질문 2: 삼성전자가 독립 경영하는 3개의 부문은 무엇인가요?
  관련 문서 ID: doc_0
  문서 내용 일부: 2
01 삼성전자 기업분석
(Samsung Electronics Co., Ltd)
Ⅰ 기업 일반 
1  기업개요
1) 기업소개 
본사주소 경기도 수원시 영통구 삼성로 129(매탄동...

질문 3: CE 부문에서 생산하는 제품에는 어떤 것들이 있나요?
  관련 문서 ID: doc_1
  문서 내용 일부: 부문 제품
CE TV, 모니터, 냉장고, 세탁기, 에어컨, 프린터, 의료기기 등
IM HHP, 네트워크시스템, 컴퓨터, 디지털카메라 등
DS DRAM, NAND Flash, 모바일...



## 4. 다양한 LangChain 리트리버 생성

이제 세 가지 다른 리트리버를 생성합니다:

1. 임베딩 리트리버:
   - 의미론적 검색이 가능한 벡터 기반 검색 방식
   - 단어가 다르더라도 의미가 비슷하면 관련 문서를 찾을 수 있음
   - 예시: "차량"을 검색하면 "자동차"가 포함된 문서를 찾을 수 있음

2. BM25 리트리버:
   - 키워드 기반의 전통적인 검색 방식
   - 정확한 단어 매칭에 강함
   - TF-IDF를 개선한 알고리즘으로, 단어 빈도와 역문서 빈도를 고려함

3. 앙상블 리트리버:
   - 임베딩과 BM25를 결합한 하이브리드 검색 방식
   - 가중치를 조정하여 두 검색 방식의 균형을 맞출 수 있음

In [ ]:
# 임베딩 모델 로드
print("임베딩 모델 로드 중...")
# HuggingFaceEmbeddings를 사용하여 BAAI/bge-m3 모델 로드
# 이 모델은 문장 임베딩에 특화된 모델로, 의미론적 검색에 좋은 성능을 보입니다
embeddings = HuggingFaceEmbeddings(model_name='BAAI/bge-m3')

임베딩 모델 로드 중...


<ipython-input-11-9a03d8c25914>:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='BAAI/bge-m3')
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [ ]:
# 임베딩 리트리버 생성
print("임베딩 리트리버 생성 중...")
# Chroma 벡터 데이터베이스를 사용하여 문서를 벡터화하고 저장
vectorstore = Chroma.from_documents(documents, embeddings)
# 벡터 저장소를 리트리버로 변환 (상위 5개 문서 검색)
embedding_retriever = vectorstore.as_retriever(search_kwargs={'k': 5})

임베딩 리트리버 생성 중...


In [ ]:
# BM25 리트리버 생성
print("BM25 리트리버 생성 중...")
# BM25 알고리즘을 사용한 리트리버 생성
bm25_retriever = BM25Retriever.from_documents(documents)
bm25_retriever.k = 5  # 상위 5개 문서 검색

BM25 리트리버 생성 중...


In [ ]:
# 앙상블 리트리버 생성
print("앙상블 리트리버 생성 중...")
# BM25와 임베딩 리트리버를 동일한 가중치(0.5:0.5)로 결합
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, embedding_retriever],
    weights=[0.5, 0.5]
)

앙상블 리트리버 생성 중...


## 4. 검색 평가 메트릭 함수

In [ ]:
# 최초 청크 분할 후, 문서에 ID 할당 코드 추가
for i, doc in enumerate(documents):
    doc.metadata['id'] = f"doc_{i}"

# 문서 ID와 내용 매핑 만들기 (평가 함수 외부에서 한 번만 실행)
document_mapping = {}
for i, doc in enumerate(documents):
    doc_id = doc.metadata['id']
    document_mapping[doc.page_content] = doc_id

# 내용으로 문서 ID 찾는 함수
def get_doc_id_by_content(content):
    """내용으로 문서 ID 찾기 (완전 일치 또는 부분 일치)"""
    # 완전 일치 확인
    if content in document_mapping:
        return document_mapping[content]

    # 부분 일치 확인 (내용이 더 길거나 짧을 수 있음)
    for doc_content, doc_id in document_mapping.items():
        if content in doc_content or doc_content in content:
            return doc_id

    return None

In [ ]:
# # 간단한 예시 문서들 생성
# from langchain.schema import Document

# # 3개의 샘플 문서 생성
# documents = [
#     Document(page_content="삼성전자는 한국의 대표적인 기업입니다.", metadata={}),
#     Document(page_content="애플은 미국의 기술 기업입니다.", metadata={}),
#     Document(page_content="인공지능은 4차 산업혁명의 핵심 기술입니다.", metadata={})
# ]

# # 1. 문서에 ID 할당
# for i, doc in enumerate(documents):
#     doc.metadata['id'] = f"doc_{i}"

# print("ID가 할당된 문서들:")
# for doc in documents:
#     print(f"ID: {doc.metadata['id']}, 내용: {doc.page_content}")

# # 2. 문서 내용-ID 매핑 생성
# document_mapping = {}
# for doc in documents:
#     doc_id = doc.metadata['id']
#     document_mapping[doc.page_content] = doc_id

# print("\n문서 내용-ID 매핑:")
# for content, doc_id in document_mapping.items():
#     print(f"내용: {content} -> ID: {doc_id}")

ID가 할당된 문서들:
ID: doc_0, 내용: 삼성전자는 한국의 대표적인 기업입니다.
ID: doc_1, 내용: 애플은 미국의 기술 기업입니다.
ID: doc_2, 내용: 인공지능은 4차 산업혁명의 핵심 기술입니다.

문서 내용-ID 매핑:
내용: 삼성전자는 한국의 대표적인 기업입니다. -> ID: doc_0
내용: 애플은 미국의 기술 기업입니다. -> ID: doc_1
내용: 인공지능은 4차 산업혁명의 핵심 기술입니다. -> ID: doc_2


In [ ]:
# # 3. get_doc_id_by_content 함수 사용 예시
# print("\nget_doc_id_by_content 함수 호출 예시:")

# # 3.1 완전 일치 케이스
# exact_content = "애플은 미국의 기술 기업입니다."
# exact_id = get_doc_id_by_content(exact_content)
# print(f"완전 일치 - 내용: '{exact_content}' -> 찾은 ID: {exact_id}")

# # 3.2 부분 일치 케이스 (더 짧은 내용)
# partial_content = "삼성전자는 한국의"
# partial_id = get_doc_id_by_content(partial_content)
# print(f"부분 일치(짧은) - 내용: '{partial_content}' -> 찾은 ID: {partial_id}")

# # 3.3 부분 일치 케이스 (더 긴 내용)
# longer_content = "인공지능은 4차 산업혁명의 핵심 기술입니다. 그리고 많은 발전이 있습니다."
# longer_id = get_doc_id_by_content(longer_content)
# print(f"부분 일치(긴) - 내용: '{longer_content}' -> 찾은 ID: {longer_id}")

# # 3.4 일치하지 않는 케이스
# no_match_content = "이 내용은 원래 문서에 없습니다."
# no_match_id = get_doc_id_by_content(no_match_content)
# print(f"불일치 - 내용: '{no_match_content}' -> 찾은 ID: {no_match_id}")


get_doc_id_by_content 함수 호출 예시:
완전 일치 - 내용: '애플은 미국의 기술 기업입니다.' -> 찾은 ID: doc_1
부분 일치(짧은) - 내용: '삼성전자는 한국의' -> 찾은 ID: doc_0
부분 일치(긴) - 내용: '인공지능은 4차 산업혁명의 핵심 기술입니다. 그리고 많은 발전이 있습니다.' -> 찾은 ID: doc_2
불일치 - 내용: '이 내용은 원래 문서에 없습니다.' -> 찾은 ID: None


### 1. Accuracy@k (정답이 상위 k개 안에 있는지 여부)

Accuracy@k는 검색 결과에서 상위 몇 개 안에 정답이 포함되었는지를 평가하는 지표입니다.  
중요한 점은 정답이 포함되기만 하면 성공으로 간주된다는 것입니다.

계산 방법:
상위 k개 결과 중 하나라도 정답이면 1, 아니면 0을 반환합니다.
여러 질문에 대해 평균을 내면 0~1 사이의 값이 됩니다.

해석 예시:
- Accuracy@5 = 0.92: 전체 질문 중 약 92%에서 상위 5개의 결과 안에 정답이 하나라도 포함
- 사용자가 상위 5개 결과만 본다면, 92%의 확률로 원하는 정보를 찾을 수 있음

실제 예시:
질문에 대해 상위 5개의 검색 결과가 다음과 같다고 가정합시다.
- 질문 1: [**정답**, 오답, 오답, 오답, 오답] → 포함 (성공)
- 질문 2: [오답, 오답, **정답**, 오답, 오답] → 포함 (성공)
- 질문 3: [오답, 오답, 오답, 오답, 오답] → 미포함 (실패)

Accuracy@5 = (1+1+0)/3 = 0.666, 즉 약 66.6%

In [ ]:
def calculate_accuracy_at_k(retrieved_docs, relevant_docs, k):
    """
    상위 k개 결과 안에 정답이 포함되어 있는지 여부를 계산합니다.

    Args:
        retrieved_docs: 검색된 문서 목록
        relevant_docs: 관련 문서 ID 집합
        k: 상위 몇 개 결과를 고려할지

    Returns:
        0 또는 1 (정답이 상위 k개 안에 없으면 0, 있으면 1)
    """
    # 상위 k개만 고려
    top_k_docs = retrieved_docs[:k] if k <= len(retrieved_docs) else retrieved_docs

    # 상위 k개 중 정답이 하나라도 있는지 확인
    for doc in top_k_docs:
        # 메타데이터에서 ID 추출 시도
        doc_id = doc.metadata.get('id')

        # 메타데이터에 ID가 없으면 내용으로 찾기
        if not doc_id:
            doc_id = get_doc_id_by_content(doc.page_content)

        # 정답에 포함되어 있는지 확인
        if doc_id in relevant_docs:
            return 1.0

    return 0.0

### 2. Precision@k (상위 k개 중 정답의 비율)

Precision@k는 상위 k개 검색 결과가 얼마나 "정확히 정답으로 이루어져 있는가?"를 평가합니다.  
상위 k개의 검색 결과 중 정답이 차지하는 비율을 측정합니다.

계산 방법:
상위 k개 중 정답의 개수를 k로 나눈 값입니다.

해석 예시:
- Precision@5 = 0.20: 상위 5개의 결과 중 평균적으로 20%가 정답
- 검색 결과에 불필요한 정보가 얼마나 적게 포함되어 있는지를 보여줌

실제 예시:
질문에 대해 상위 5개의 검색 결과가 다음과 같다고 가정합시다.
- 질문 1: [**정답**, 오답, 오답, 오답, 오답] → Precision@5 = 1/5 = 0.2
- 질문 2: [오답, **정답**, 오답, 오답, 오답] → Precision@5 = 1/5 = 0.2
- 질문 3: [오답, 오답, 오답, 오답, 오답] → Precision@5 = 0/5 = 0.0

평균 Precision@5 = (0.2 + 0.2 + 0.0)/3 = 0.133

In [ ]:
def calculate_precision_at_k(retrieved_docs, relevant_docs, k):
    """
    상위 k개 결과 중 정답의 비율을 계산합니다.

    Args:
        retrieved_docs: 검색된 문서 목록
        relevant_docs: 관련 문서 ID 집합
        k: 상위 몇 개 결과를 고려할지

    Returns:
        0~1 사이의 값 (상위 k개 중 정답의 비율)
    """
    # 상위 k개만 고려
    top_k_docs = retrieved_docs[:k] if k <= len(retrieved_docs) else retrieved_docs

    if not top_k_docs:
        return 0.0

    # 상위 k개 중 정답의 개수
    relevant_count = 0
    for doc in top_k_docs:
        # 메타데이터에서 ID 추출 시도
        doc_id = doc.metadata.get('id')

        # 메타데이터에 ID가 없으면 내용으로 찾기
        if not doc_id:
            doc_id = get_doc_id_by_content(doc.page_content)

        # 정답에 포함되어 있는지 확인
        if doc_id in relevant_docs:
            relevant_count += 1

    return relevant_count / len(top_k_docs)

### 3. Recall@k (전체 정답 중 상위 k개에 포함된 정답의 비율)

Recall@k는 검색 결과가 얼마나 "포괄적으로" 정답을 포함하고 있는지를 평가합니다.  
전체 정답 중 검색 결과 상위 k개 안에 포함된 정답의 비율을 나타냅니다.

계산 방법:
상위 k개에 포함된 정답의 개수를 전체 정답 개수로 나눈 값입니다.  
각 질문당 정답이 하나씩만 있는 경우 Recall과 Accuracy의 값이 동일합니다.

해석 예시:
- Recall@5 = 0.7은 전체 정답 중 70%가 상위 5개 결과에 포함되었다는 의미입니다.
- 높을수록 검색 시스템이 관련 문서를 빠짐없이 찾아낸다는 뜻입니다.
- 중요한 정보를 놓치지 않아야 하는 상황에서 중요한 지표입니다.
- 모든 관련 정보를 찾는 능력을 평가합니다.

실제 예시 (질문 하나에 정답이 두 개인 경우):
- 질문 1: [**정답**, **정답**, 오답, 오답, 오답] → Recall@5 = 2/2 = 1.0
- 질문 2: [**정답**, 오답, 오답, 오답, 오답] → Recall@5 = 1/2 = 0.5
- 질문 3: [오답, 오답, 오답, 오답, 오답] → Recall@5 = 0/2 = 0.0

평균 Recall@5 = (1.0 + 0.5 + 0.0)/3 = 0.5

In [ ]:
def calculate_recall_at_k(retrieved_docs, relevant_docs, k):
    """
    전체 정답 중 상위 k개에 포함된 정답의 비율을 계산합니다.

    Args:
        retrieved_docs: 검색된 문서 목록
        relevant_docs: 관련 문서 ID 집합
        k: 상위 몇 개 결과를 고려할지

    Returns:
        0~1 사이의 값 (전체 정답 중 상위 k개에 포함된 비율)
    """
    if not relevant_docs:
        return 0.0

    # 상위 k개만 고려
    top_k_docs = retrieved_docs[:k] if k <= len(retrieved_docs) else retrieved_docs

    # 상위 k개에 포함된 정답의 개수 계산
    relevant_found = 0
    for doc in top_k_docs:
        # 메타데이터에서 ID 추출 시도
        doc_id = doc.metadata.get('id')

        # 메타데이터에 ID가 없으면 내용으로 찾기
        if not doc_id:
            doc_id = get_doc_id_by_content(doc.page_content)

        # 정답에 포함되어 있는지 확인
        if doc_id in relevant_docs:
            relevant_found += 1

    return relevant_found / len(relevant_docs)

### 4. MRR@k (Mean Reciprocal Rank - 첫 번째 정답 순위의 역수)

MRR@k는 첫 번째 정답이 등장한 순위의 역수를 계산합니다.
첫 정답이 상위에 있을수록 높은 점수를 부여합니다.

계산 방법:
- 정답이 1위에 있으면 MRR = 1.0
- 정답이 2위에 있으면 MRR = 0.5
- 정답이 3위에 있으면 MRR = 0.33...
- 상위 k개 안에 정답이 없으면 MRR = 0

해석 예시:
- MRR@10 = 0.5: 첫 번째 정답이 평균적으로 2위에 등장한다는 의미
- 사용자가 정답을 얼마나 "빨리" 찾을 수 있는지를 평가하는 지표

실제 예시:
질문에 대해 상위 10개의 검색 결과가 다음과 같다고 가정합니다.
- 질문 1: [**정답**, 오답, 오답, ...] Reciprocal Rank = 1/1 = 1.0
- 질문 2: [오답, **정답**, 오답, ...] Reciprocal Rank = 1/2 = 0.5
- 질문 3: [오답, 오답, **정답**, ...] Reciprocal Rank = 1/3 = 0.33
- 질문 4: [오답, 오답, 오답, ...] Reciprocal Rank = 0 (정답이 상위 10위 안에 없음)

평균 MRR@10 = (1.0 + 0.5 + 0.33 + 0)/4 = 0.458

In [ ]:
def calculate_mrr_at_k(retrieved_docs, relevant_docs, k):
    """
    Mean Reciprocal Rank를 계산합니다 (첫 번째 정답 순위의 역수).

    Args:
        retrieved_docs: 검색된 문서 목록
        relevant_docs: 관련 문서 ID 집합
        k: 상위 몇 개 결과를 고려할지

    Returns:
        0~1 사이의 값 (첫 번째 정답 순위의 역수, 없으면 0)
    """
    # 상위 k개만 고려
    top_k_docs = retrieved_docs[:k] if k <= len(retrieved_docs) else retrieved_docs

    # 첫 번째 정답 순위 찾기
    for i, doc in enumerate(top_k_docs):
        # 메타데이터에서 ID 추출 시도
        doc_id = doc.metadata.get('id')

        # 메타데이터에 ID가 없으면 내용으로 찾기
        if not doc_id:
            doc_id = get_doc_id_by_content(doc.page_content)

        # 정답에 포함되어 있는지 확인
        if doc_id in relevant_docs:
            return 1.0 / (i + 1)  # 0부터 시작하므로 +1

    return 0.0

### 5. NDCG@k (Normalized Discounted Cumulative Gain)

NDCG@k는 검색 결과의 관련성과 순위를 모두 고려하는 메트릭입니다.  
관련 문서가 높은 순위에 배치될수록 높은 점수를 부여합니다.

계산 방법:
1. DCG(Discounted Cumulative Gain) 계산:
   - 검색된 각 문서가 정답인지 확인합니다.
   - 정답인 문서에 대해, 위치에 따라 할인된 점수를 부여합니다(log₂(rank+1)로 나눔).
   - 이 점수들의 합을 구합니다.

2. Ideal DCG 계산:
   - 모든 정답 문서가 상위에 있는 이상적인 경우의 DCG를 계산합니다.

3. NDCG = DCG / Ideal DCG (0~1 사이의 값)

해석 예시:
- NDCG@10 = 0.85: 검색 결과가 이상적인 순서에 85% 근접함
- 정답의 순위 분포를 고려하는 종합적인 메트릭

실제 예시:
- 질문 1: [정답, 정답, 오답] → NDCG = 1.0 (정답이 모두 상위에 있음)
- 질문 2: [오답, 정답, 오답] → NDCG ≈ 0.63 (정답이 두 번째 위치에 있음)
- 질문 3: [오답, 오답, 정답] → NDCG ≈ 0.39 (정답이 세 번째 위치에 있음)

평균 NDCG@3 = (1.0 + 0.63 + 0.39)/3 ≈ 0.673

In [ ]:
def calculate_ndcg_at_k(retrieved_docs, relevant_docs, k):
    """
    NDCG@k (Normalized Discounted Cumulative Gain)를 계산합니다.

    Args:
        retrieved_docs: 검색된 문서 목록
        relevant_docs: 관련 문서 ID 집합
        k: 상위 몇 개 결과를 고려할지

    Returns:
        0~1 사이의 값 (1에 가까울수록 이상적인 검색 결과)
    """
    # 상위 k개만 고려
    top_k_docs = retrieved_docs[:k] if k <= len(retrieved_docs) else retrieved_docs

    # 실제 DCG 계산
    dcg = 0
    for i, doc in enumerate(top_k_docs):
        # 메타데이터에서 ID 추출 시도
        doc_id = doc.metadata.get('id')

        # 메타데이터에 ID가 없으면 내용으로 찾기
        if not doc_id:
            doc_id = get_doc_id_by_content(doc.page_content)

        # 정답에 포함되어 있는지 확인
        if doc_id in relevant_docs:
            # i는 0부터 시작하므로 i+1이 실제 순위
            dcg += 1.0 / np.log2(i + 2)  # log_2(rank + 1)

    # 이상적인 DCG 계산 (모든 관련 문서가 상위에 있을 경우)
    ideal_dcg = 0
    for i in range(min(len(relevant_docs), k)):
        ideal_dcg += 1.0 / np.log2(i + 2)

    # NDCG 계산
    return dcg / ideal_dcg if ideal_dcg > 0 else 0.0

### 6. MAP@k (Mean Average Precision)

MAP@k는 각 정답을 찾을 때마다의 Precision 값을 계산하여 평균을 낸 값입니다.
검색 결과의 전반적인 정확도와 일관성을 평가합니다.

계산 방법:
1. 각 정답 발견 시 해당 위치까지의 Precision 계산
2. 이 Precision 값들의 평균 산출

해석 예시:
- MAP@100 = 0.818: 상위 100개 내에서 정답을 찾을 때마다 계산된 Precision의 평균이 0.818
- 검색 시스템의 전반적인 일관성을 측정하는 종합적인 지표

실제 예시:
상위 5개 결과가 [**정답**, 오답, **정답**, 오답, **정답**]인 경우:
- 첫 번째 정답 발견 시 Precision = 1/1 = 1.0
- 두 번째 정답 발견 시 Precision = 2/3 = 0.67
- 세 번째 정답 발견 시 Precision = 3/5 = 0.6
- MAP@5 = (1.0 + 0.67 + 0.6) / 3 = 0.76

In [ ]:
def calculate_map_at_k(retrieved_docs, relevant_docs, k):
    """
    MAP@k (Mean Average Precision)를 계산합니다.

    Args:
        retrieved_docs: 검색된 문서 목록
        relevant_docs: 관련 문서 ID 집합
        k: 상위 몇 개 결과를 고려할지

    Returns:
        0~1 사이의 값 (각 정답 발견 시 precision의 평균)
    """
    # 상위 k개만 고려
    top_k_docs = retrieved_docs[:k] if k <= len(retrieved_docs) else retrieved_docs

    # 각 정답을 찾을 때마다의 Precision 계산
    precisions = []
    relevant_count = 0

    for i, doc in enumerate(top_k_docs):
        # 메타데이터에서 ID 추출 시도
        doc_id = doc.metadata.get('id')

        # 메타데이터에 ID가 없으면 내용으로 찾기
        if not doc_id:
            doc_id = get_doc_id_by_content(doc.page_content)

        # 정답에 포함되어 있는지 확인
        if doc_id in relevant_docs:
            relevant_count += 1
            precisions.append(relevant_count / (i + 1))

    # MAP 계산
    return sum(precisions) / len(relevant_docs) if precisions and relevant_docs else 0.0

In [ ]:
# 모든 메트릭을 계산하는 함수
def calculate_all_metrics(retrieved_docs, relevant_docs, k_values=[1, 3, 5, 10]):
    """
    모든 평가 메트릭을 계산하는 함수

    Args:
        retrieved_docs: 검색된 문서 목록
        relevant_docs: 관련 문서 ID 집합
        k_values: 평가할 k 값들의 리스트

    Returns:
        모든 메트릭 결과를 담은 딕셔너리
    """
    metrics = {}

    # 각 k 값에 대해 모든 메트릭 계산
    for k in k_values:
        metrics[f'Accuracy@{k}'] = calculate_accuracy_at_k(retrieved_docs, relevant_docs, k)
        metrics[f'Precision@{k}'] = calculate_precision_at_k(retrieved_docs, relevant_docs, k)
        metrics[f'Recall@{k}'] = calculate_recall_at_k(retrieved_docs, relevant_docs, k)
        metrics[f'MRR@{k}'] = calculate_mrr_at_k(retrieved_docs, relevant_docs, k)
        metrics[f'NDCG@{k}'] = calculate_ndcg_at_k(retrieved_docs, relevant_docs, k)
        metrics[f'MAP@{k}'] = calculate_map_at_k(retrieved_docs, relevant_docs, k)

    return metrics

## 5. 리트리버 평가 함수

In [ ]:
def evaluate_retriever(retriever, queries, relevant_docs, name="", k_values=[1, 3, 5, 10]):
    """
    각 리트리버의 성능을 평가하는 함수

    Args:
        retriever: 평가할 리트리버 객체
        queries: 질문 ID를 키로, 질문 텍스트를 값으로 하는 딕셔너리
        relevant_docs: 질문 ID를 키로, 관련 문서 ID 집합을 값으로 하는 딕셔너리
        name: 리트리버 이름 (출력용)
        k_values: 평가할 k 값들의 리스트

    Returns:
        평균 메트릭 딕셔너리
    """
    print(f"\n{name} 리트리버 평가 중...")
    results = []

    # 일부 질문만 평가 (속도 향상을 위해)
    # 실제 평가에서는 모든 질문을 사용하는 것이 더 정확합니다
    sample_queries = dict(list(queries.items())[:20])

    # 각 질문에 대해 리트리버 평가
    for query_id, query_text in tqdm(sample_queries.items()):
        # 리트리버로 문서 검색
        retrieved_docs = retriever.get_relevant_documents(query_text)

        # 관련 문서 ID 가져오기
        expected_ids = relevant_docs.get(query_id, set())

        # 검색 결과 평가 (모든 메트릭 계산)
        metrics = calculate_all_metrics(retrieved_docs, expected_ids, k_values)

        # 결과 저장
        result = {
            'query_id': query_id,
            'query': query_text,
            **metrics
        }
        results.append(result)

    # 데이터프레임으로 변환
    df_results = pd.DataFrame(results)

    # 평균 메트릭 계산
    metrics_columns = [col for col in df_results.columns if any(col.startswith(prefix) for prefix in
                                                              ['Accuracy', 'Precision', 'Recall', 'MRR', 'NDCG', 'MAP'])]
    avg_metrics = df_results[metrics_columns].mean().to_dict()

    # 주요 메트릭 출력
    print(f"{name} 리트리버 평가 결과 (평균):")
    for k in sorted(k_values):
        print(f"  k={k} 결과:")
        for metric_prefix in ['Accuracy', 'Precision', 'Recall', 'MRR', 'NDCG', 'MAP']:
            metric_key = f"{metric_prefix}@{k}"
            if metric_key in avg_metrics:
                print(f"    {metric_key}: {avg_metrics[metric_key]:.4f}")

    return avg_metrics

In [ ]:
# 각 리트리버 평가
print("\n리트리버 평가 시작...")
embedding_metrics = evaluate_retriever(embedding_retriever, queries, relevant_docs, "임베딩")
bm25_metrics = evaluate_retriever(bm25_retriever, queries, relevant_docs, "BM25")
ensemble_metrics = evaluate_retriever(ensemble_retriever, queries, relevant_docs, "앙상블")


리트리버 평가 시작...

임베딩 리트리버 평가 중...


  0%|          | 0/20 [00:00<?, ?it/s]

<ipython-input-23-46e66528470d>:25: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(query_text)


임베딩 리트리버 평가 결과 (평균):
  k=1 결과:
    Accuracy@1: 0.8500
    Precision@1: 0.8500
    Recall@1: 0.8500
    MRR@1: 0.8500
    NDCG@1: 0.8500
    MAP@1: 0.8500
  k=3 결과:
    Accuracy@3: 0.9500
    Precision@3: 0.3167
    Recall@3: 0.9500
    MRR@3: 0.9000
    NDCG@3: 0.9131
    MAP@3: 0.9000
  k=5 결과:
    Accuracy@5: 1.0000
    Precision@5: 0.2000
    Recall@5: 1.0000
    MRR@5: 0.9125
    NDCG@5: 0.9346
    MAP@5: 0.9125
  k=10 결과:
    Accuracy@10: 1.0000
    Precision@10: 0.2000
    Recall@10: 1.0000
    MRR@10: 0.9125
    NDCG@10: 0.9346
    MAP@10: 0.9125

BM25 리트리버 평가 중...


  0%|          | 0/20 [00:00<?, ?it/s]

BM25 리트리버 평가 결과 (평균):
  k=1 결과:
    Accuracy@1: 0.6500
    Precision@1: 0.6500
    Recall@1: 0.6500
    MRR@1: 0.6500
    NDCG@1: 0.6500
    MAP@1: 0.6500
  k=3 결과:
    Accuracy@3: 0.8000
    Precision@3: 0.2667
    Recall@3: 0.8000
    MRR@3: 0.7250
    NDCG@3: 0.7446
    MAP@3: 0.7250
  k=5 결과:
    Accuracy@5: 0.8500
    Precision@5: 0.1700
    Recall@5: 0.8500
    MRR@5: 0.7350
    NDCG@5: 0.7640
    MAP@5: 0.7350
  k=10 결과:
    Accuracy@10: 0.8500
    Precision@10: 0.1700
    Recall@10: 0.8500
    MRR@10: 0.7350
    NDCG@10: 0.7640
    MAP@10: 0.7350

앙상블 리트리버 평가 중...


  0%|          | 0/20 [00:00<?, ?it/s]

앙상블 리트리버 평가 결과 (평균):
  k=1 결과:
    Accuracy@1: 0.7500
    Precision@1: 0.7500
    Recall@1: 0.7500
    MRR@1: 0.7500
    NDCG@1: 0.7500
    MAP@1: 0.7500
  k=3 결과:
    Accuracy@3: 0.9500
    Precision@3: 0.3167
    Recall@3: 0.9500
    MRR@3: 0.8417
    NDCG@3: 0.8696
    MAP@3: 0.8417
  k=5 결과:
    Accuracy@5: 1.0000
    Precision@5: 0.2000
    Recall@5: 1.0000
    MRR@5: 0.8542
    NDCG@5: 0.8912
    MAP@5: 0.8542
  k=10 결과:
    Accuracy@10: 1.0000
    Precision@10: 0.1308
    Recall@10: 1.0000
    MRR@10: 0.8542
    NDCG@10: 0.8912
    MAP@10: 0.8542


## 6. 결과 비교 및 분석

In [ ]:
results = {
    '임베딩': embedding_metrics,
    'BM25': bm25_metrics,
    '앙상블': ensemble_metrics
}

# 데이터프레임으로 변환 (전치하여 메트릭을 행으로 변환)
results_df = pd.DataFrame(results)

print("\n리트리버 성능 비교:")
print(results_df)

# 결과 저장
results_df.to_csv("retriever_comparison_results.csv")
print("평가 결과를 'retriever_comparison_results.csv'에 저장했습니다.")


리트리버 성능 비교:
                   임베딩      BM25       앙상블
Accuracy@1    0.850000  0.650000  0.750000
Precision@1   0.850000  0.650000  0.750000
Recall@1      0.850000  0.650000  0.750000
MRR@1         0.850000  0.650000  0.750000
NDCG@1        0.850000  0.650000  0.750000
MAP@1         0.850000  0.650000  0.750000
Accuracy@3    0.950000  0.800000  0.950000
Precision@3   0.316667  0.266667  0.316667
Recall@3      0.950000  0.800000  0.950000
MRR@3         0.900000  0.725000  0.841667
NDCG@3        0.913093  0.744639  0.869639
MAP@3         0.900000  0.725000  0.841667
Accuracy@5    1.000000  0.850000  1.000000
Precision@5   0.200000  0.170000  0.200000
Recall@5      1.000000  0.850000  1.000000
MRR@5         0.912500  0.735000  0.854167
NDCG@5        0.934627  0.763982  0.891173
MAP@5         0.912500  0.735000  0.854167
Accuracy@10   1.000000  0.850000  1.000000
Precision@10  0.200000  0.170000  0.130754
Recall@10     1.000000  0.850000  1.000000
MRR@10        0.912500  0.735000  0.85416

## 7. 실제 출력 비교

In [ ]:
def analyze_all_retrievers(queries, corpus, relevant_docs, bm25_retriever, embedding_retriever, ensemble_retriever, num_samples=5, top_k=3):
    """
    세 가지 리트리버(BM25, 임베딩, 앙상블)의 검색 결과를 비교 분석하는 함수

    Args:
        queries: 질문 ID를 키로, 질문 텍스트를 값으로 하는 딕셔너리
        corpus: 문서 ID를 키로, 문서 텍스트를 값으로 하는 딕셔너리
        relevant_docs: 질문 ID를 키로, 관련 문서 ID 집합을 값으로 하는 딕셔너리
        bm25_retriever: BM25 리트리버 객체
        embedding_retriever: 임베딩 리트리버 객체
        ensemble_retriever: 앙상블 리트리버 객체
        num_samples: 분석할 질문 수
        top_k: 출력할 상위 검색 결과 수
    """
    print(f"\n===== 리트리버 검색 결과 비교 분석 (샘플 {num_samples}개) =====\n")

    # 샘플 질문 선택
    sample_query_ids = list(queries.keys())[:num_samples]

    # 내용으로 문서 ID 찾는 함수
    def find_doc_id_by_content(content):
        for doc_id, doc_content in corpus.items():
            if content == doc_content or content in doc_content or doc_content in content:
                return doc_id
        return None

    # 각 질문에 대해 분석
    for idx, query_id in enumerate(sample_query_ids):
        query_text = queries[query_id]
        expected_doc_ids = relevant_docs.get(query_id, set())

        print(f"\n{'='*80}")
        print(f"\n[질문 {idx+1}] {query_text}")
        print('--' * 20)

        # 정답 문서 내용 출력
        print("\n💡 정답 문서:")
        for doc_id in expected_doc_ids:
            doc_text = corpus.get(doc_id, "문서를 찾을 수 없음")
            print(f"  문서 ID: {doc_id}")
            print(f"  내용: {doc_text[:150]}..." if len(doc_text) > 150 else doc_text)

        # 각 리트리버별 검색 결과 출력
        retrievers = {
            "BM25": bm25_retriever,
            "임베딩": embedding_retriever,
            "앙상블": ensemble_retriever
        }
        print('==' * 50)
        for name, retriever in retrievers.items():
            # 검색 수행
            retrieved_docs = retriever.get_relevant_documents(query_text)

            # 검색 결과 출력
            print(f"\n🔍 {name} 리트리버 검색 결과 (상위 {top_k}개):")
            for i, doc in enumerate(retrieved_docs[:top_k]):
                # 문서 ID 확인 (메타데이터 또는 내용 기반)
                doc_id = doc.metadata.get('id', None)
                if not doc_id or doc_id.startswith("unknown"):
                    doc_id = find_doc_id_by_content(doc.page_content) or f"unknown_{i}"

                is_relevant = doc_id in expected_doc_ids
                relevance_mark = "✓" if is_relevant else "✗"

                print(f"  [{i+1}] {relevance_mark} 문서 ID: {doc_id}")
                print(f"      내용: {doc.page_content[:150]}..." if len(doc.page_content) > 150 else doc.page_content)
                print('--' * 20)

            # 정확도 계산
            correct_in_top_k = 0
            for doc in retrieved_docs[:top_k]:
                doc_id = doc.metadata.get('id', None)
                if not doc_id or doc_id.startswith("unknown"):
                    doc_id = find_doc_id_by_content(doc.page_content)

                if doc_id in expected_doc_ids:
                    correct_in_top_k += 1

            accuracy = correct_in_top_k / min(top_k, len(retrieved_docs)) if retrieved_docs else 0
            print(f"  정확도: {correct_in_top_k}/{min(top_k, len(retrieved_docs))} ({accuracy:.2f})")

        # 키워드 분석
        print("\n🔑 질문-문서 키워드 분석:")
        query_words = set(query_text.lower().split())

        # 정답 문서의 키워드
        answer_doc_id = next(iter(expected_doc_ids)) if expected_doc_ids else None
        if answer_doc_id:
            answer_doc_text = corpus.get(answer_doc_id, "")
            answer_words = set(answer_doc_text.lower().split())
            common_words = query_words.intersection(answer_words)

            print(f"  질문 키워드: {', '.join(query_words)}")
            print(f"  정답 문서와 공통 키워드: {', '.join(common_words)}")

        print(f"\n{'='*80}")

# 함수 실행
analyze_all_retrievers(
    queries,
    corpus,
    relevant_docs,
    bm25_retriever,
    embedding_retriever,
    ensemble_retriever
)


===== 리트리버 검색 결과 비교 분석 (샘플 5개) =====



[질문 1] 삼성전자의 설립일은 언제인가요?
----------------------------------------

💡 정답 문서:
  문서 ID: doc_0
  내용: 2
01 삼성전자 기업분석
(Samsung Electronics Co., Ltd)
Ⅰ 기업 일반 
1  기업개요
1) 기업소개 
본사주소 경기도 수원시 영통구 삼성로 129(매탄동 416)
사업분야 삼성그룹의 대표 기업으로 휴대폰, 정보통신기기, 반도체, TV 등을 생...

🔍 BM25 리트리버 검색 결과 (상위 3개):
  [1] ✗ 문서 ID: doc_39
자본총계 105,888 122,371 132,677 136,428
2015년말 기준 삼성전자의 총자산 금액은 약 169조에 달하며, 부채는 32.5조, 자본총계는 136조에 이르고 
있다.
----------------------------------------
  [2] ✗ 문서 ID: doc_32
      내용: 15
✜ 신성장동력 → 사물인터넷(Internet of Things, IoT)
삼성전자는 다가올 사물인터넷(IoT, Internet of Things) 시대를 맞이해 만반의 준비를 다하고 있습
니다. 2014년 8월에 미국의 ‘스마트싱스(SmartThings)’를 인수...
----------------------------------------
  [3] ✗ 문서 ID: doc_25
      내용: 12
Q2 삼성전자의 Harman사 인수에 대해 어떻게 생각하십니까 ?
A
 삼성전자가 최근 80억달러(약 9조)에 미국의 전장업체(자동차 전자기기에 대한 사업)인 Harman 사를 
인수하기로 결정했습니다. 하만사는 세계적으로 자동차용 인포테인먼트와 텔레매틱스 시장에...
----------------------------------------
  정확도: 0/3 (0.00)

🔍 임베딩 리트리버 검색 결과 (상위 3개):
  [1] ✓ 문서 I